In [6]:
import pandas as pd

df=pd.read_excel('1000_dummy_product.xlsx')
df.head()

,ID,Deskripsi Produk,Merek,Kategori,Harga Sewa,Preferensi Pengguna,Jumlah Stok
0,1,Kamera Mirrorless,Sony,Camera,200000,"Fotografi, Ringan, Kinerja",5
1,2,Speaker Bluetooth,JBL,Speaker,100000,"Bergaya, Portabel",3
2,3,Proyektor Mini,Epson,Proyektor,150000,"Kinerja, Mudah Digunakan",2
3,4,Playstation 5,Sony,Playstation,250000,"Mewah, Gaming",4
4,5,Tenda Camping 4 Orang,Coleman,Tenda,50000,"Nyaman, Ringan, Tahan Lama",6


In [7]:
for column in df.Kategori:
    df['Kategori'] = df['Kategori'].replace('Camera', 'Kamera')
    df['Kategori'] = df['Kategori'].replace('Kamera Aksi', 'Kamera')
    df['Kategori'] = df['Kategori'].replace('Tas Camping', 'Tas')

df.Kategori.value_counts()

Speaker                140
Kamera                 124
Proyektor              105
Tenda                  104
Sepatu                 104
Matras                 103
Tas                    103
Playstation            101
Dapur                   22
Alat Musik              12
Laptop                  12
Perkakas                 8
Outdoor                  7
Smartphone               7
Furniture                7
Smart TV                 5
Alat Fitness             4
Drone                    4
Monitor                  4
Mesin Jahit              3
Kulkas                   3
Printer                  2
Perangkat Streaming      2
Mikrofon                 2
Headset                  2
Videocam                 1
Audio                    1
Koper                    1
Perangkat Keamanan       1
Soundbar                 1
TV                       1
Sepeda                   1
Keyboard                 1
Mesin Cuci               1
Smartwatch               1
Name: Kategori, dtype: int64

In [8]:
# Menghitung jumlah data per kategori
kategori_counts = df['Kategori'].value_counts()

# Mencari kategori yang memiliki jumlah data kurang dari 5
kategori_kurang_dari_5 = kategori_counts[kategori_counts < 10].index

# Menghapus baris yang memiliki kategori yang jumlahnya kurang dari 5
df = df[~df['Kategori'].isin(kategori_kurang_dari_5)]


In [9]:
# Menghapus atribut "ID" dan "Jumlah Stok"
df1 = df.drop(['ID'], axis=1)
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 930 entries, 0 to 999
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Deskripsi Produk     930 non-null    object
 1   Merek                930 non-null    object
 2   Kategori             930 non-null    object
 3   Harga Sewa           930 non-null    int64 
 4   Preferensi Pengguna  930 non-null    object
 5   Jumlah Stok          930 non-null    int64 
dtypes: int64(2), object(4)
memory usage: 50.9+ KB


In [10]:
from sklearn.preprocessing import MinMaxScaler
# Inisialisasi MinMaxScaler
scaler = MinMaxScaler()

# Mengambil kolom "Harga Sewa" dan mengubahnya menjadi bentuk array
harga_sewa = df1['Harga Sewa'].values.reshape(-1, 1)

# Melakukan Min-Max Scaling pada kolom "Harga Sewa"
harga_sewa_scaled = scaler.fit_transform(harga_sewa)

# Mengganti nilai pada kolom "Harga Sewa" dengan nilai yang sudah di-scaling
df1['Harga Sewa'] = harga_sewa_scaled

df1.head()

,Deskripsi Produk,Merek,Kategori,Harga Sewa,Preferensi Pengguna,Jumlah Stok
0,Kamera Mirrorless,Sony,Kamera,0.008513,"Fotografi, Ringan, Kinerja",5
1,Speaker Bluetooth,JBL,Speaker,0.003505,"Bergaya, Portabel",3
2,Proyektor Mini,Epson,Proyektor,0.006009,"Kinerja, Mudah Digunakan",2
3,Playstation 5,Sony,Playstation,0.011017,"Mewah, Gaming",4
4,Tenda Camping 4 Orang,Coleman,Tenda,0.001002,"Nyaman, Ringan, Tahan Lama",6


In [12]:
from sklearn.preprocessing import LabelEncoder

# Inisialisasi LabelEncoder
label_encoder = LabelEncoder()

# Encoding atribut "Merek"
df1['Merek'] = label_encoder.fit_transform(df1['Merek'])

# # Encoding atribut "Kategori"
# df['Kategori'] = label_encoder.fit_transform(df['Kategori'])
# df['Deskripsi Produk'] = label_encoder.fit_transform(df['Deskripsi Produk'])
# df['Preferensi Pengguna'] = label_encoder.fit_transform(df['Preferensi Pengguna'])

df1.head()


,Deskripsi Produk,Merek,Kategori,Harga Sewa,Preferensi Pengguna,Jumlah Stok
0,Kamera Mirrorless,93,Kamera,0.008513,"Fotografi, Ringan, Kinerja",5
1,Speaker Bluetooth,48,Speaker,0.003505,"Bergaya, Portabel",3
2,Proyektor Mini,30,Proyektor,0.006009,"Kinerja, Mudah Digunakan",2
3,Playstation 5,93,Playstation,0.011017,"Mewah, Gaming",4
4,Tenda Camping 4 Orang,22,Tenda,0.001002,"Nyaman, Ringan, Tahan Lama",6


In [13]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [15]:
# Inisialisasi WordNetLemmatizer dan set stopwords
lemmatizer = WordNetLemmatizer()

# Fungsi untuk melakukan preprocessing teks
def preprocess_text(text):
    # Mengubah huruf menjadi lowercase
    text = text.lower()
    # Menghapus tanda baca
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenisasi kata-kata
    tokens = nltk.word_tokenize(text)
    # Lemmatisasi kata-kata
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    # Menggabungkan kembali kata-kata menjadi kalimat
    processed_text = ' '.join(lemmatized_tokens)
    return processed_text

# Standarisasi teks pada atribut "Deskripsi Produk"
df1['Deskripsi Produk'] = df1['Deskripsi Produk'].apply(preprocess_text)

# Standarisasi teks pada atribut "Preferensi Pengguna"
df1['Preferensi Pengguna'] = df1['Preferensi Pengguna'].apply(preprocess_text)


In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse as sp
# Inisialisasi TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Representasi TF-IDF untuk fitur "Deskripsi Produk"
deskripsi_produk_tfidf = tfidf_vectorizer.fit_transform(df1['Deskripsi Produk'])

# Representasi TF-IDF untuk fitur "Preferensi Pengguna"
preferensi_pengguna_tfidf = tfidf_vectorizer.fit_transform(df1['Preferensi Pengguna'])

# Menggabungkan kedua matriks representasi menjadi satu matriks
feature_matrix = sp.hstack((deskripsi_produk_tfidf, preferensi_pengguna_tfidf))



In [24]:
import pandas as pd

# Membuat one-hot encoding dari atribut 'Kategori'
one_hot_encoded = pd.get_dummies(df1['Kategori'])

# Menggabungkan one-hot encoded dengan DataFrame asli
df1 = pd.concat([df1, one_hot_encoded], axis=1)

# Menghapus kolom 'Kategori' yang tidak perlu lagi setelah di one-hot encoding
df1 = df1.drop('Kategori', axis=1)


In [25]:
df1.head()

,Deskripsi Produk,Merek,Harga Sewa,Preferensi Pengguna,Jumlah Stok,Alat Musik,Dapur,Kamera,Laptop,Matras,Playstation,Proyektor,Sepatu,Speaker,Tas,Tenda
0,kamera mirrorless,93,0.008513,fotografi ringan kinerja,5,0,0,1,0,0,0,0,0,0,0,0
1,speaker bluetooth,48,0.003505,bergaya portabel,3,0,0,0,0,0,0,0,0,1,0,0
2,proyektor mini,30,0.006009,kinerja mudah digunakan,2,0,0,0,0,0,0,1,0,0,0,0
3,playstation 5,93,0.011017,mewah gaming,4,0,0,0,0,0,1,0,0,0,0,0
4,tenda camping 4 orang,22,0.001002,nyaman ringan tahan lama,6,0,0,0,0,0,0,0,0,0,0,1


In [37]:
from sklearn.model_selection import train_test_split

target_data = df1.drop(['Deskripsi Produk', 'Merek', 'Harga Sewa', 'Preferensi Pengguna', 'Jumlah Stok'], axis=1)

# Pisahkan data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(feature_matrix, target_data, test_size=0.2, random_state=42)

In [38]:
feature_matrix.shape

(930, 481)

In [39]:
target_data.shape

(930, 11)

In [42]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense

# Memisahkan data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(feature_matrix.toarray(), target_data.to_numpy(), test_size=0.2, random_state=42)

# Definisikan jumlah fitur input
input_dim = X_train.shape[1]

# Inisialisasi model
model = Sequential()

# Tambahkan layer input dan layer tersembunyi
model.add(Dense(64, input_dim=input_dim, activation='relu'))
model.add(Dense(32, activation='relu'))

# Tambahkan layer output dengan jumlah kelas yang sesuai
num_classes = y_train.shape[1]
model.add(Dense(num_classes, activation='softmax'))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Melatih model dengan data latih
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluasi model dengan data uji
evaluation = model.evaluate(X_test, y_test)
print("Accuracy:", evaluation[1])


Epoch 1/10
24/24 [==============================] - 2s 24ms/step - loss: 2.3012 - accuracy: 0.4274 - val_loss: 2.1739 - val_accuracy: 0.6452
Epoch 2/10
24/24 [==============================] - 0s 5ms/step - loss: 2.0013 - accuracy: 0.6720 - val_loss: 1.7629 - val_accuracy: 0.7204
Epoch 3/10
24/24 [==============================] - 0s 5ms/step - loss: 1.4794 - accuracy: 0.8199 - val_loss: 1.1743 - val_accuracy: 0.8280
Epoch 4/10
24/24 [==============================] - 0s 4ms/step - loss: 0.8836 - accuracy: 0.9274 - val_loss: 0.6692 - val_accuracy: 0.9194
Epoch 5/10
24/24 [==============================] - 0s 4ms/step - loss: 0.4518 - accuracy: 0.9503 - val_loss: 0.3842 - val_accuracy: 0.9462
Epoch 6/10
24/24 [==============================] - 0s 4ms/step - loss: 0.2503 - accuracy: 0.9503 - val_loss: 0.2678 - val_accuracy: 0.9516
Epoch 7/10
24/24 [==============================] - 0s 5ms/step - loss: 0.1663 - accuracy: 0.9677 - val_loss: 0.2087 - val_accuracy: 0.9516
Epoch 8/10
24/24 [=

In [43]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Prediksi nilai menggunakan model
train_pred = model.predict(X_train)
test_pred = model.predict(X_test)

# Menghitung MSE dan MAE
train_mse = mean_squared_error(y_train, train_pred)
test_mse = mean_squared_error(y_test, test_pred)

train_mae = mean_absolute_error(y_train, train_pred)
test_mae = mean_absolute_error(y_test, test_pred)

print("MSE pada data latih: ", train_mse)
print("MSE pada data uji: ", test_mse)

print("MAE pada data latih: ", train_mae)
print("MAE pada data uji: ", test_mae)


6/6 [==============================] - 0s 1ms/step
MSE pada data latih:  0.0012714275
MSE pada data uji:  0.0038896734
MAE pada data latih:  0.007647741
MAE pada data uji:  0.013135248
